In [1]:
import pandas as pd
import spacy
#import en_core_web_md

ModuleNotFoundError: No module named 'pandas'

In [ ]:
train_data = pd.read_csv('data/QA_train_data.csv')
test_data = pd.read_csv('data/QA_test_data.csv')

In [ ]:
test_data.head(20)

In [ ]:
nlp = spacy.load("en_core_web_lg")

In [ ]:
train_data = train_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

In [ ]:
def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove stopwords
    text = " ".join([token for token in text.split() if token not in nlp.Defaults.stop_words])
    # Lemmatize the text
    text = " ".join([token.lemma_ for token in nlp(text)])
    return text

# Apply the preprocessing to the questions
train_data['Question'] = train_data['Question'].apply(preprocess_text)
test_data['Question'] = test_data['Question'].apply(preprocess_text)

In [ ]:
def predict_answer(question: str)-> str:
    # Convert the question to a Spacy Doc object
    question_doc = nlp(question)

    # Convert the questions to Spacy Doc objects
    faq_docs = list(nlp.pipe(train_data['Question']))

    # Compute the similarity of the question to each Previous question
    similarities = [question_doc.similarity(faq_doc) for faq_doc in faq_docs]

    # Find the index of the most similar Previous question
    max_index = similarities.index(max(similarities))

    # If the similarity is above a certain threshold, return the corresponding Previous answer
    if max(similarities) > 0.9:  # Increase the threshold
        return train_data['Answer'].iloc[max_index]
    else:
        return "No Answer."

In [ ]:
predicted = test_data['Question'].apply(lambda q: predict_answer(q))
actual = test_data['Answer']

In [ ]:
def evaluate_result(predicted: str, actual: str) -> int:
    if predicted is None:
        return 0
    elif predicted == actual:
        return 1
    else:return -1

def evaluate_All_results(predicted: pd.Series, actual: pd.Series) -> float:
    total = 0
    for index, value in predicted.items():
        eval = evaluate_result(value, actual[index])
        total += eval
    result = total / predicted.size
    return result

In [ ]:
score = evaluate_All_results(predicted, actual)

In [ ]:
print(score)